In [2]:
from rich.console import Console
import sys
import builtins

console = Console(force_terminal=False, file=sys.__stdout__)

builtins.print = console.print

In [3]:
texto = """
A Microsoft foi fundada em 4 de abril de 1975 por Bill Gates e Paul Allen, em Albuquerque, 
no estado do Novo México. O nome "Microsoft" é uma combinação das palavras "microcomputador" 
e "software", refletindo o foco da empresa em software para computadores pessoais.
O primeiro grande projeto da Microsoft foi a criação de um sistema operacional para o 
computador Altair 8800, um dos primeiros microcomputadores disponíveis comercialmente. 
O sistema, denominado Altair BASIC, foi desenvolvido em parceria com a MITS 
(Micro Instrumentation and Telemetry Systems) e foi um marco inicial para a Microsoft.
Em 1980, a empresa firmou um contrato significativo com a IBM para fornecer o sistema 
operacional para o novo PC da IBM, o que levou à criação do MS-DOS. Esse contrato foi um 
ponto de virada para a Microsoft, impulsionando sua expansão e dominando o mercado de 
sistemas operacionais para PCs nos anos seguintes.
Com o sucesso do MS-DOS, a Microsoft se consolidou como líder no setor de software e, 
em 1985, lançou o Windows, um sistema operacional gráfico que viria a se tornar a base 
de sua supremacia no mercado de sistemas operacionais para desktop.
"""

In [4]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function

In [5]:
class Event(BaseModel):
    """Informações sobre um evento ocorrido"""
    date: str = Field(description="Data do evento no formato YYYY-MM-DD")
    event: str = Field(description="Descrição do evento extraído do texto")
    
class EventsList(BaseModel):
    """Lista de Eventos para Extração"""
    events: List[Event] = Field(description="Conjunto de eventos encontrados no texto fornecido")
    
tool_events = convert_to_openai_function(EventsList)
print(tool_events)

{
    'name': 'EventsList',
    'description': 'Lista de Eventos para Extração',
    'parameters': {
        'properties': {
            'events': {
                'description': 'Conjunto de eventos encontrados no texto fornecido',
                'items': {
                    'description': 'Informações sobre um evento ocorrido',
                    'properties': {
                        'date': {'description': 'Data do evento no formato YYYY-MM-DD', 'type': 'string'},
                        'event': {'description': 'Descrição do evento extraído do texto', 'type': 'string'}
                    },
                    'required': ['date', 'event'],
                    'type': 'object'
                },
                'type': 'array'
            }
        },
        'required': ['events'],
        'type': 'object'
    }
}

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extraia as frases de acontecimentos e as extraia integralmente"),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_events],
                            function_call={"name":"EventsList"}))

In [7]:
print(chain.invoke({"input": texto}))

AIMessage(
    content='',
    additional_kwargs={
        'function_call': {
            'arguments': '{"events":[{"date":"1975-04-04","event":"A Microsoft foi fundada por Bill Gates e Paul 
Allen em Albuquerque, Novo México."},{"date":"1980","event":"A Microsoft firmou um contrato com a IBM para fornecer
o sistema operacional para o novo PC da IBM, resultando no desenvolvimento do MS-DOS."},{"date":"1985","event":"A 
Microsoft lançou o Windows, um sistema operacional gráfico que se tornou a base de sua supremacia no mercado de 
sistemas operacionais para desktop."}]}',
            'name': 'EventsList'
        },
        'refusal': None
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 116,
            'prompt_tokens': 426,
            'total_tokens': 542,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo-0125',
        'system_fingerprint': None,
        'id': 'chatcmpl-BrMV4gj7F2fNjLOp95XZJZT69mqvh',
        'service_tier': 'default',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--fefc54c1-9b92-442d-8623-637a1c8102b9-0',
    usage_metadata={
        'input_tokens': 426,
        'output_tokens': 116,
        'total_tokens': 542,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 0}
    }
)

In [8]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (prompt 
         | chat.bind(functions=[tool_events], function_call={'name': 'EventsList'})
         | JsonOutputFunctionsParser())

print(chain.invoke({'input': texto}))

{
    'events': [
        {
            'date': '1975-04-04',
            'event': 'A Microsoft foi fundada por Bill Gates e Paul Allen em Albuquerque, Novo México.'
        },
        {
            'date': '1980',
            'event': 'A Microsoft firmou contrato com a IBM para fornecer o sistema operacional para o novo PC, 
levando à criação do MS-DOS.'
        },
        {'date': '1985', 'event': 'A Microsoft lançou o Windows, um sistema operacional gráfico.'}
    ]
}

In [9]:
from langchain_community.document_loaders.web_base import WebBaseLoader

loader = WebBaseLoader("https://www.techtudo.com.br")
page = loader.load()
print(page)

USER_AGENT environment variable not set, consider setting it to identify your requests.


[
    Document(
        metadata={
            'source': 'https://www.techtudo.com.br',
            'title': 'TechTudo: tecnologia, celular, computador e games',
            'description': 'O TechTudo reúne as principais notícias de tecnologia, reviews de celulares, TVs e 
computadores, além de dicas sobre aparelhos, aplicativos e softwares.',
            'language': 'pt-BR'
        },
        page_content=' TechTudo: tecnologia, celular, computador e games                top e últimas       Voltar 
top e últimas    últimas do techtudo     notícias     comparativos     dicas e tutoriais     guias     listas     
reviews     vídeos     tudo sobre     webstories       celulares e tablets       Voltar     celulares e tablets    
notícias de celulares     celulares     tablets     telefonia     smartwatches     e-reader     android     iOS    
fórum     --     iPhone 14     iphone 13     galaxy s22 ultra     moto g stylus     redmi note 10       jogos      
Voltar     jogos    notícias de jogos     jogos casuais     jogos de esportes     jogos de terror     mais jogos   
Voltar     mais jogos    jogos de ação     jogos arcade     jogos de aventura     jogos de corrida     jogos de 
estratégia     jogos de luta     jogos de raciocínio     jogos de rpg     jogos de tabuleiro     jogos educativos  
playstation     xbox     nintendo     fórum     --     gta 5     roblox     minecraft       esports       Voltar   
esports    notícias de esports     campeonatos     times     atletas     --     cblol     liga brasileira de free 
fire     twitch tv       informática       Voltar     informática    notícias de informática     computadores      
Voltar     computadores    pc     pc gamer       notebooks       Voltar     notebooks    notebooks     notebooks 2 
em 1     notebook gamer       impressoras       Voltar     impressoras    impressoras     impressoras 3d       
mouses     placas e componentes     roteadores     --     windows 11     macos     linux       eletrodomésticos    
Voltar     eletrodomésticos    air fryer     fogões     geladeiras     máquinas de lavar       eletrônicos       
Voltar     eletrônicos    notícias de eletrônicos     ar-condicionado     caixas de som     câmeras       Voltar   
câmeras    câmeras     filmadoras       carros smart     fones de ouvido     máquinas de cartão     tv box     TVs 
veículos     --     gopro hero      chromecast     amazon echo dot       softwares       Voltar     softwares    
notícias de softwares     aplicativos para celular     áudio e vídeo     imagens     inteligência artificial     
internet     produtividade     redes sociais     segurança     utilitários     --     whatsapp     instagram     
facebook     youtube       Streaming       Voltar     Streaming    notícias de streaming     Animes     Séries     
Voltar     Séries    dorama     séries       Filmes     Jogos de futebol     Programas de TV     Plataformas       
--     tech 60+     prime day     qual comprar     iphone 16     --     quem somos     fórum techtudo     canal do 
tt no whatsapp     CBN tecnologia     conteúdo de marca       Voltar     conteúdo de marca    americanas           
princípios editoriais        termos de uso        política de privacidade                         Menu             
TechTudo                                                               últimas     aplicativos     celulares     
eletrônicos     informática     eletrodomésticos     jogos     softwares     streaming     qual comprar         
Publicidade        \n\n\n\n\n                      FIQUE POR DENTRO\n                    \n                    
CANAL DO TECHTUDO NO WHATSAPP\n                  \n\n\n\n                      DICAS, REVIEWS E MAIS\n             
\n\n\n\n\n\n\n\n\n\n      MAIOR DA HISTÓRIA? GTA 6 pode ter orçamento de R$ 5,4 bilhões; veja rumores Jogo da 
Rockstar pode ser o mais caro da história        MESTRE DO CHATGPT Essas dicas vão deixar suas imagens feitas com 
IA perfeitas        CUIDADO! Esses sites vão en

In [10]:
class BlogPost(BaseModel):
    """Detalhes sobre uma postagem de blog"""
    title: str = Field(description="Título da postagem no blog")
    author: str = Field(description="Nome do autor da postagem no blog")
    
class BlogSite(BaseModel):
    """Conjunto de postagens de blog de um site específico"""
    posts: List[BlogPost] = Field(description="Coleção de postagens de blog do site")
    
tool_blog = convert_to_openai_function(BlogSite)
print(tool_blog)

{
    'name': 'BlogSite',
    'description': 'Conjunto de postagens de blog de um site específico',
    'parameters': {
        'properties': {
            'posts': {
                'description': 'Coleção de postagens de blog do site',
                'items': {
                    'description': 'Detalhes sobre uma postagem de blog',
                    'properties': {
                        'title': {'description': 'Título da postagem no blog', 'type': 'string'},
                        'author': {'description': 'Nome do autor da postagem no blog', 'type': 'string'}
                    },
                    'required': ['title', 'author'],
                    'type': 'object'
                },
                'type': 'array'
            }
        },
        'required': ['posts'],
        'type': 'object'
    }
}

In [11]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extraia da página os posts do blog com as informações especificadas'),
    ('user', '{input}')
])
chat = ChatOpenAI()
chain = (prompt 
         | chat.bind(functions=[tool_blog], function_call={'name': 'BlogSite'})
         | JsonKeyOutputFunctionsParser(key_name='posts'))

In [12]:
print(chain.invoke({"input": page}))

[
    {
        'title': 'MAIOR DA HISTÓRIA? GTA 6 pode ter orçamento de R$ 5,4 bilhões; veja rumores',
        'author': 'TechTudo'
    },
    {
        'title': 'MESTRE DO CHATGPT Essas dicas vão deixar suas imagens feitas com IA perfeitas',
        'author': 'TechTudo'
    },
    {'title': 'CUIDADO! Esses sites vão encher o seu computador de vírus; evite', 'author': 'TechTudo'},
    {
        'title': 'OFERTAS! O que ficar de olho no Prime Day? Tudo que você precisa saber para economizar!',
        'author': 'TechTudo'
    },
    {
        'title': 'Melhores antivírus Grátis para celular Android! Veja o Top 3 atualizado para 2025!',
        'author': 'TechTudo'
    }
]